In [1]:
import json
import os

import pandas as pd

In [2]:
with open('./data/ml_energy/models.json', 'r') as fd:
    model_params = json.load(fd)

In [3]:
all_dfs = []

for root, _, files in os.walk("./data/ml_energy"):
    for file in files:
        if 'benchmark' in file and file.endswith('.csv'):
            gpu, task, _ = file.split('_')
            df = pd.read_csv(os.path.join(root, file))
            df['gpu'] = gpu
            df['task'] = task
            df['parameters_count'] = df.model.apply(lambda x: model_params[x]['params'])
            all_dfs.append(df)

df = pd.concat(all_dfs)

In [4]:
# convert Joules to Wh
df['energy_per_token'] = (df['energy'] / df['response_length']) / 3600

In [5]:
df['dtype'] = 'float16'

In [6]:
df['optimization'] = 'None'

In [7]:
df['quantization'] = 'None'

In [8]:
df['cuda-fp16'] = False

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 260 entries, 0 to 20
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   model             260 non-null    object 
 1   throughput        260 non-null    float64
 2   response_length   260 non-null    float64
 3   latency           260 non-null    float64
 4   energy            260 non-null    float64
 5   gpu               260 non-null    object 
 6   task              260 non-null    object 
 7   parameters_count  260 non-null    int64  
 8   energy_per_token  260 non-null    float64
 9   dtype             260 non-null    object 
 10  optimization      260 non-null    object 
 11  quantization      260 non-null    object 
 12  cuda-fp16         260 non-null    bool   
dtypes: bool(1), float64(5), int64(1), object(6)
memory usage: 26.7+ KB


In [10]:
df.to_csv('./data/aggregated_ml_energy.csv', index=False)